__DQ checks for Final Task. RAW data__

In [1]:
import pandas as pd
#DataFrame that contains test results
result_table = pd.DataFrame(columns = ['Table','DQ check','Column','Status','Bad Data'])


__Function for check completeness by not null value__

__TC.1 Carrier: completeness by not nullable fields__

In [2]:
def Uniqueness_by_PK(path,list_of_columns):
    data = sqlContext.read.parquet(path).toPandas()
    result_row = []
    a = []
    col = []
    col_count = len(list_of_columns)
    for i in range (col_count):
        a.append(data[list_of_columns[i]].value_counts())
    res_count = pd.DataFrame(a)
    len_col = len(res_count.columns)
    for i in range(len(res_count)):
        for j in res_count.columns:
            col.append(j)
            if res_count[j][i] > 1:
                status = 'Failed'
                bad_data = f'Exist duplicates fro value {j}. Duplicates count is {res_count[j][i]}.'
            else:
                continue
            result_row.append([path.split('.csv')[0].title(),'Uniqueness', res_count.index[i],status,bad_data])
    res = pd.DataFrame(result_row)
    res.columns = ['Table','DQ check','Column','Status','Bad Data']
    return res

path_airports = pAirports='gs://iskldl04-dqelearn-local-bucket/raw/airports'
list_of_columns_airports = ['iata']
#list_of_columns_airports = ['iata','city']
result_airports = Uniqueness_by_PK(path_airports,list_of_columns_airports)
print(result_airports)

                                              Table    DQ check Column  \
0  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Airports  Uniqueness   iata   
1  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Airports  Uniqueness   iata   
2  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Airports  Uniqueness   iata   
3  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Airports  Uniqueness   iata   
4  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Airports  Uniqueness   iata   
5  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Airports  Uniqueness   iata   
6  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Airports  Uniqueness   iata   

   Status                                           Bad Data  
0  Failed  Exist duplicates fro value 00V. Duplicates cou...  
1  Failed  Exist duplicates fro value 00R. Duplicates cou...  
2  Failed  Exist duplicates fro value . Duplicates count ...  
3  Failed  Exist duplicates fro value Z13. Duplicates cou...  
4  Failed  Exist duplicates fro value Z09. Duplicates cou...  
5  Failed  Exist duplicates f

__Function for check Uniqueness by PK__ 

__TC2. Airports: Uniqueness by PK__ 

In [3]:
def Completeness_by_not_null(path,list_of_columns):
    data = sqlContext.read.parquet(path).toPandas()
    result_row = []
    a = []
    col_count = len(list_of_columns)
    for i in range (col_count):
        a.append(data[list_of_columns[i]].isnull().sum())
        if a[i] > 0:
            status = 'Failed'
            bad_data = f'In the column {list_of_columns[i]} exist {a[i]} rows with Null value.'
        else:
            status = 'Passed'
            bad_data = f'In the column  {list_of_columns[i]} no null value.'
        result_row.append([path.split('.csv')[0].title(),'Completeness',list_of_columns[i],status,bad_data])
    res = pd.DataFrame(result_row)
    res.columns = ['Table','DQ check','Column','Status','Bad Data']
    return res

list_of_columns_carriers = ['code','description']
path_carriers = 'gs://iskldl04-dqelearn-local-bucket/raw/carriers'
result_carriers = Completeness_by_not_null(path_carriers,list_of_columns_carriers)
print(result_carriers)

                                              Table      DQ check  \
0  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Carriers  Completeness   
1  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/Carriers  Completeness   

        Column  Status                                   Bad Data  
0         code  Passed         In the column  code no null value.  
1  description  Passed  In the column  description no null value.  


__Function for  Consistency check for Foreign Key__

__TC3. Flights: Consistency check for UniqueCarrier__

In [4]:
def Consistency_by_FK(path_pk,col_pk,path_fk,col_fk):
    car_code = sqlContext.read.parquet(path_pk).toPandas()
    air_uniqcar =  sqlContext.read.parquet(path_fk).toPandas() 
    car_code_array = car_code[col_pk].to_numpy()
    air_uniqcar_array = air_uniqcar[col_fk].drop_duplicates().to_numpy()
    result_row = []
    for i in range(len(air_uniqcar_array)):
        if air_uniqcar_array[i] not in car_code_array:
            status = 'Failed'
            col = col_fk[0]
            colpk = col_pk[0]
            value = air_uniqcar_array[i][0]
            bad_data = f'In the column {col} exist value {value} not from {colpk}'
            result_row.append([path_fk.split('.csv')[0].title(),'Consistency by FK', col, status,bad_data])
        else:
            continue
    res = pd.DataFrame(result_row)
    res.columns = ['Table','DQ check','Column','Status','Bad Data']
    return res

path_flights = 'gs://iskldl04-dqelearn-local-bucket/raw/ flights'
column_flights_fk = ['UniqueCarrier']
column_carriers_pk = ['code']

result_flights_fk = Consistency_by_FK(path_carriers,column_carriers_pk,path_flights,column_flights_fk)
print(result_flights_fk)

22/12/12 18:15:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


                                              Table           DQ check  \
0  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency by FK   
1  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency by FK   

          Column  Status                                           Bad Data  
0  UniqueCarrier  Failed  In the column UniqueCarrier exist value AXX no...  
1  UniqueCarrier  Failed  In the column UniqueCarrier exist value 1B9 no...  


__Function for  Flights: Consistency check for Time Delay__

__TC4. Flights: Consistency check for DepDelay__    
Calculated column: DepTime - CRSDepTime (mins)

In [5]:
def Consistency_TimeDelay(path,time_col_agg, agg_1,agg_2):
    data = sqlContext.read.parquet(path).toPandas()
    result_row = []
    data['difference'] = data[agg_1[0]].replace(['NA',''],[0,0]).astype(int) - data[agg_2[0]].replace(['NA',''],[0,0]).astype(int)
    for i in range(len(data)):
        if data[time_col_agg[0]][i] != data['difference'][i]:
            aggres = data['difference']
            status = 'Failed'
            bad_data = f'In the row {i} data do not match computed calue based on other attributes. {data[time_col_agg[0]][i]} = {time_col_agg[0]} != {agg_1[0]} - {agg_2[0]} = {aggres[i]}'
            result_row.append([path.split('.csv')[0].title(),'Consistency DepDelay', time_col_agg[0], status, bad_data])
        else:
            continue
    res = pd.DataFrame(result_row)
    res.columns = ['Table','DQ check','Column','Status','Bad Data']
    return res

col = ['DepDelay']
col1 = ['DepTime']
col2 = ['CRSDepTime']
result_flight_time = Consistency_TimeDelay(path_flights,col,col1,col2)
print(result_flight_time)

                                                Table              DQ check  \
0    Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
1    Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
2    Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
3    Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
4    Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
..                                                ...                   ...   
532  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
533  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
534  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
535  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   
536  Gs://Iskldl04-Dqelearn-Local-Bucket/Raw/ Flights  Consistency DepDelay   

       Column  Status                              

__Saving test results in the csv file__

In [6]:
result = pd.concat([result_table,result_carriers, result_airports, result_flights_fk, result_flight_time], ignore_index=True)
result.index += 1

result.to_csv('Test_result_Raw.csv')